<a href="https://colab.research.google.com/github/bwowby/DS/blob/master/ir-basic/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word Embedding
희소(sparse) 벡터인 원핫벡터와는 달리 단어를 밀집(dense) 벡터로 표현하는 방법.   
LSA,Word2Vec,FastText,Glove 등의 방법론이 있다.


### Word2Vec

영어/한국어 Word2Vec 실습   
https://wikidocs.net/50739

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
# xml로 되어있는 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7efec02eae10>)